In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import snakeviz
%load_ext snakeviz

In [ ]:
import os
import sys
from math import log

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.kohonen_som import KohonenSOM

som_class = KohonenSOM(10,10,3,11)

# Create random data set with 10 entries
rand = np.random.RandomState(0)
#train_data = rand.randint(0, 255, (20, 3))
train_data = np.random.random((20,3))


# Train SOM   
%prun -D program.prof som_class.train_som(train_data, 2000)

fig, ax = plt.subplots(
    nrows=1, ncols=3, figsize=(12, 3.5), 
    subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow(train_data.reshape(5,4,3), vmin=0, vmax=1)
ax[0].title.set_text('Train Data')
ax[1].imshow(som_class.som_grid_initial, vmin=0, vmax=1)
ax[1].title.set_text('Som Initial Data')
ax[2].imshow(som_class.som_grid, vmin=0, vmax=1)
ax[2].title.set_text('Som trained')


In [ ]:
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

data = np.genfromtxt('../resources/data.csv', delimiter=',', usecols=(0,1,2,3), skip_header=1)
#data = (data - np.min(data)) / (np.max(data) - np.min(data))
data = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, data)
labels = np.genfromtxt('../resources/data.csv', delimiter=',', usecols=(4), dtype=str, skip_header=1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, stratify=labels, test_size=0.20)

som_class = KohonenSOM(7,7,4,3,0.5)
som_class.train_som(X_train, 2000)

winmap = defaultdict(list)
for x, l in zip(X_train, y_train):
    winmap[som_class.find_bmu(x)].append(l)
for position in winmap:
    winmap[position] = Counter(winmap[position])

default_class = np.sum(list(winmap.values())).most_common()[0][0]

def classify(som: KohonenSOM, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    
    result = []
    for d in data:
        win_position = som.find_bmu(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

print(classification_report(y_test, classify(som_class, X_test)))

